# Portfolio Optimization with Tech Stocks

This notebook demonstrates portfolio optimization techniques using Modern Portfolio Theory (MPT) to find the optimal allocation of tech stocks.

## Overview

We'll cover:
1. Fetching historical data for tech stocks
2. Calculating returns and risk metrics
3. Finding the efficient frontier
4. Identifying the optimal portfolio (maximum Sharpe ratio)
5. Visualizing results

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 1. Fetch Historical Data

We'll analyze a portfolio of major tech stocks: Apple, Microsoft, Google, Amazon, and NVIDIA.

In [ ]:
# Define tech stocks portfolio
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA']

# Set date range (2 years of data)
end_date = datetime.now()
start_date = end_date - timedelta(days=2*365)

# Download data
print(f"Fetching data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Display first few rows
print(f"\nData shape: {data.shape}")
data.head()

## 2. Calculate Returns and Risk Metrics

In [ ]:
# Calculate daily returns
returns = data.pct_change().dropna()

# Calculate annualized metrics
annual_returns = returns.mean() * 252
annual_std = returns.std() * np.sqrt(252)
correlation_matrix = returns.corr()
covariance_matrix = returns.cov() * 252

# Display statistics
stats_df = pd.DataFrame({
    'Annual Return': annual_returns,
    'Annual Volatility': annual_std,
    'Sharpe Ratio': annual_returns / annual_std
})

print("Individual Stock Statistics:")
print(stats_df.round(4))

## 3. Visualize Correlation Matrix

In [ ]:
# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Tech Stocks', fontsize=16)
plt.tight_layout()
plt.show()

## 4. Portfolio Optimization Functions

In [ ]:
def portfolio_metrics(weights, returns, cov_matrix):
    """
    Calculate portfolio return, volatility, and Sharpe ratio
    """
    portfolio_return = np.dot(weights, returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = portfolio_return / portfolio_volatility
    return portfolio_return, portfolio_volatility, sharpe_ratio

def negative_sharpe(weights, returns, cov_matrix):
    """
    Negative Sharpe ratio for minimization
    """
    return -portfolio_metrics(weights, returns, cov_matrix)[2]

def portfolio_volatility(weights, returns, cov_matrix):
    """
    Portfolio volatility for minimization
    """
    return portfolio_metrics(weights, returns, cov_matrix)[1]

## 5. Generate Efficient Frontier

In [ ]:
# Number of assets
n_assets = len(tickers)

# Constraints and bounds
constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}  # weights sum to 1
bounds = tuple((0, 1) for _ in range(n_assets))  # no short selling

# Generate efficient frontier
frontier_returns = []
frontier_volatility = []
frontier_weights = []

# Target returns for efficient frontier
target_returns = np.linspace(annual_returns.min(), annual_returns.max(), 100)

for target in target_returns:
    # Add return constraint
    cons = [constraints,
            {'type': 'eq', 'fun': lambda x, target=target: portfolio_metrics(x, annual_returns, covariance_matrix)[0] - target}]
    
    # Minimize volatility for given return
    result = minimize(portfolio_volatility, 
                     x0=np.array(n_assets * [1. / n_assets]),
                     args=(annual_returns, covariance_matrix),
                     method='SLSQP',
                     bounds=bounds,
                     constraints=cons)
    
    if result.success:
        ret, vol, _ = portfolio_metrics(result.x, annual_returns, covariance_matrix)
        frontier_returns.append(ret)
        frontier_volatility.append(vol)
        frontier_weights.append(result.x)

## 6. Find Optimal Portfolios

In [ ]:
# Maximum Sharpe Ratio Portfolio
max_sharpe = minimize(negative_sharpe,
                     x0=np.array(n_assets * [1. / n_assets]),
                     args=(annual_returns, covariance_matrix),
                     method='SLSQP',
                     bounds=bounds,
                     constraints=constraints)

max_sharpe_weights = max_sharpe.x
max_sharpe_return, max_sharpe_vol, max_sharpe_ratio = portfolio_metrics(max_sharpe_weights, annual_returns, covariance_matrix)

# Minimum Volatility Portfolio
min_vol = minimize(portfolio_volatility,
                  x0=np.array(n_assets * [1. / n_assets]),
                  args=(annual_returns, covariance_matrix),
                  method='SLSQP',
                  bounds=bounds,
                  constraints=constraints)

min_vol_weights = min_vol.x
min_vol_return, min_vol_vol, min_vol_ratio = portfolio_metrics(min_vol_weights, annual_returns, covariance_matrix)

# Display optimal portfolios
print("Maximum Sharpe Ratio Portfolio:")
print(f"Return: {max_sharpe_return:.2%}")
print(f"Volatility: {max_sharpe_vol:.2%}")
print(f"Sharpe Ratio: {max_sharpe_ratio:.4f}")
print("\nWeights:")
for ticker, weight in zip(tickers, max_sharpe_weights):
    if weight > 0.01:
        print(f"{ticker}: {weight:.2%}")

print("\n" + "="*50)
print("\nMinimum Volatility Portfolio:")
print(f"Return: {min_vol_return:.2%}")
print(f"Volatility: {min_vol_vol:.2%}")
print(f"Sharpe Ratio: {min_vol_ratio:.4f}")
print("\nWeights:")
for ticker, weight in zip(tickers, min_vol_weights):
    if weight > 0.01:
        print(f"{ticker}: {weight:.2%}")

## 7. Visualize Efficient Frontier

In [ ]:
# Create efficient frontier plot
plt.figure(figsize=(12, 8))

# Plot efficient frontier
plt.plot(frontier_volatility, frontier_returns, 'b-', linewidth=3, label='Efficient Frontier')

# Plot individual stocks
for i, ticker in enumerate(tickers):
    plt.scatter(annual_std[ticker], annual_returns[ticker], s=200, label=ticker)

# Plot optimal portfolios
plt.scatter(max_sharpe_vol, max_sharpe_return, marker='*', color='red', s=500, 
           label=f'Max Sharpe ({max_sharpe_ratio:.2f})')
plt.scatter(min_vol_vol, min_vol_return, marker='*', color='green', s=500,
           label=f'Min Volatility')

# Formatting
plt.xlabel('Volatility (Standard Deviation)', fontsize=14)
plt.ylabel('Expected Return', fontsize=14)
plt.title('Efficient Frontier - Tech Stocks Portfolio', fontsize=16)
plt.legend(loc='best', fontsize=12)
plt.grid(True, alpha=0.3)

# Format axes as percentages
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.0%}'.format(x)))

plt.tight_layout()
plt.show()

## 8. Portfolio Allocation Visualization

In [ ]:
# Create pie charts for optimal portfolios
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Max Sharpe Portfolio
weights_sharpe = [w for w in max_sharpe_weights if w > 0.01]
labels_sharpe = [tickers[i] for i, w in enumerate(max_sharpe_weights) if w > 0.01]
ax1.pie(weights_sharpe, labels=labels_sharpe, autopct='%1.1f%%', startangle=90)
ax1.set_title(f'Maximum Sharpe Ratio Portfolio\n(Sharpe: {max_sharpe_ratio:.2f})', fontsize=14)

# Min Volatility Portfolio
weights_vol = [w for w in min_vol_weights if w > 0.01]
labels_vol = [tickers[i] for i, w in enumerate(min_vol_weights) if w > 0.01]
ax2.pie(weights_vol, labels=labels_vol, autopct='%1.1f%%', startangle=90)
ax2.set_title(f'Minimum Volatility Portfolio\n(Vol: {min_vol_vol:.1%})', fontsize=14)

plt.tight_layout()
plt.show()

## 9. Monte Carlo Simulation

In [ ]:
# Run Monte Carlo simulation
num_portfolios = 10000
results = np.zeros((3, num_portfolios))

np.random.seed(42)
for i in range(num_portfolios):
    # Generate random weights
    weights = np.random.random(n_assets)
    weights /= np.sum(weights)  # normalize to sum to 1
    
    # Calculate metrics
    ret, vol, sharpe = portfolio_metrics(weights, annual_returns, covariance_matrix)
    
    results[0, i] = ret
    results[1, i] = vol
    results[2, i] = sharpe

# Create scatter plot
plt.figure(figsize=(12, 8))
plt.scatter(results[1], results[0], c=results[2], cmap='viridis', alpha=0.5)
plt.colorbar(label='Sharpe Ratio')

# Plot efficient frontier
plt.plot(frontier_volatility, frontier_returns, 'r-', linewidth=3, label='Efficient Frontier')

# Plot optimal portfolios
plt.scatter(max_sharpe_vol, max_sharpe_return, marker='*', color='red', s=500, 
           label='Max Sharpe', edgecolors='black', linewidth=2)
plt.scatter(min_vol_vol, min_vol_return, marker='*', color='green', s=500,
           label='Min Volatility', edgecolors='black', linewidth=2)

plt.xlabel('Volatility (Standard Deviation)', fontsize=14)
plt.ylabel('Expected Return', fontsize=14)
plt.title('Monte Carlo Simulation - 10,000 Random Portfolios', fontsize=16)
plt.legend(loc='best', fontsize=12)
plt.grid(True, alpha=0.3)

# Format axes as percentages
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.0%}'.format(x)))

plt.tight_layout()
plt.show()

## 10. Summary and Recommendations

Based on our analysis:

1. **Diversification Benefits**: The correlation matrix shows that while tech stocks are somewhat correlated, there are still diversification benefits

2. **Optimal Portfolios**:
   - **Maximum Sharpe Ratio**: Best risk-adjusted returns
   - **Minimum Volatility**: Lowest risk portfolio

3. **Key Insights**:
   - The efficient frontier shows the trade-off between risk and return
   - No single stock dominates the optimal portfolios
   - Portfolio optimization can significantly improve risk-adjusted returns

4. **Next Steps**:
   - Consider adding constraints (e.g., maximum allocation per stock)
   - Include transaction costs in the optimization
   - Perform out-of-sample testing
   - Add more asset classes for better diversification